# DRC Roads Temporal Analysis



Keywords: temporal analysis, forest monitoring, classification

In [6]:
import datetime
import json
import os

import numpy as np
import pandas as pd
from planet import api

In [7]:
# Import functionality from local notebooks
from ipynb.fs.defs.drc_roads_download import get_overlapping_scenes, StripDownloader
from ipynb.fs.defs.drc_roads_classification \
    import Timer, load_training_data, classify_forest, classified_band_to_rgb

## Download Scenes

In this section, we use the ipynb module to import `StripDownloader` from the [drc_roads_download](drc_roads_download.ipynb) notebook. We then use `StripDownloader` to download the portion of OrthoTile strips that overlap the AOI.

### Load data from drc_roads_download notebook

In [8]:
# load aoi. Saved in drc_roads_download notebook
aoi_geojson_filename = os.path.join('pre-data', 'aoi.geojson')

In [9]:
# # load scenes database. Saved in drc_roads_download notebook
# overlapping_scenes_filename = os.path.join('pre-data', 'overlapping-scenes')
# overlapping_scenes = pd.read_pickle(overlapping_scenes_filename)

### Download portions of OrthoTile strips that overlap AOI

In [5]:
# create planet api client
planet_api_key = os.environ['PL_API_KEY']

# quick check that key is defined
assert planet_api_key, "PL_API_KEY not defined."

client = api.ClientV1(api_key=planet_api_key)

In [5]:
# load aoi json
with open(aoi_geojson_filename) as json_data:
    aoi = json.load(json_data)

NameError: name 'json' is not defined

In [8]:
item_type = 'PSOrthoTile'

In [9]:
# get overlapping scenes from to July 2017 to February 2018
begin=datetime.datetime(year=2017,month=7,day=1)
end=datetime.datetime(year=2018,month=3,day=1)
overlapping_scenes = get_overlapping_scenes(aoi, begin, end, client, item_type)

500 scenes with intersecting borders
494 scenes with intersecting footprints
179 intersecting strips
36 strips with significant overlap
There are 144 OrthoTiles in 36 strips that significantly overlap the aoi


In [14]:
asset_type = 'analytic'
strip_downloader = StripDownloader(overlapping_scenes, aoi_geojson_filename, client)
strip_downloader.run(asset_type, overwrite=False, verbose=True)

36 strips
found data/1214499_mosaic.tif
found data/1176905_mosaic.tif
found data/1167439_mosaic.tif
found data/1153620_mosaic.tif
found data/1150329_mosaic.tif
found data/1130991_mosaic.tif
found data/1127986_mosaic.tif
found data/1124454_mosaic.tif
found data/1121892_mosaic.tif
found data/1114901_mosaic.tif
found data/1043874_mosaic.tif
found data/1091588_mosaic.tif
found data/1085449_mosaic.tif
found data/1081381_mosaic.tif
found data/1054596_mosaic.tif
found data/1047845_mosaic.tif
found data/1043827_mosaic.tif
found data/1037433_mosaic.tif
found data/1011123_mosaic.tif
found data/993122_mosaic.tif
found data/960830_mosaic.tif
found data/943459_mosaic.tif
found data/904538_mosaic.tif
found data/915538_mosaic.tif
found data/883193_mosaic.tif
found data/879726_mosaic.tif
found data/863467_mosaic.tif
found data/758681_mosaic.tif
found data/741529_mosaic.tif
found data/739199_mosaic.tif
found data/705233_mosaic.tif
found data/699281_mosaic.tif
found data/651530_mosaic.tif
found data/666

### Get mosaic image names

In [32]:
# sort strip ids as integers. Strip id order is the same as strip temporal order
strip_ids = [int(s) for s in overlapping_scenes.strip_id.unique().tolist()]
strip_ids.sort()
strip_ids = [str(s) for s in strip_ids]

In [33]:
strip_mosaics = [strip_downloader.get_filename(i) for i in strip_ids]

In [34]:
strip_mosaics

['data/624045_mosaic.tif',
 'data/636659_mosaic.tif',
 'data/651530_mosaic.tif',
 'data/666617_mosaic.tif',
 'data/699281_mosaic.tif',
 'data/705233_mosaic.tif',
 'data/739199_mosaic.tif',
 'data/741529_mosaic.tif',
 'data/758681_mosaic.tif',
 'data/863467_mosaic.tif',
 'data/879726_mosaic.tif',
 'data/883193_mosaic.tif',
 'data/904538_mosaic.tif',
 'data/915538_mosaic.tif',
 'data/943459_mosaic.tif',
 'data/960830_mosaic.tif',
 'data/993122_mosaic.tif',
 'data/1011123_mosaic.tif',
 'data/1037433_mosaic.tif',
 'data/1043827_mosaic.tif',
 'data/1043874_mosaic.tif',
 'data/1047845_mosaic.tif',
 'data/1054596_mosaic.tif',
 'data/1081381_mosaic.tif',
 'data/1085449_mosaic.tif',
 'data/1091588_mosaic.tif',
 'data/1114901_mosaic.tif',
 'data/1121892_mosaic.tif',
 'data/1124454_mosaic.tif',
 'data/1127986_mosaic.tif',
 'data/1130991_mosaic.tif',
 'data/1150329_mosaic.tif',
 'data/1153620_mosaic.tif',
 'data/1167439_mosaic.tif',
 'data/1176905_mosaic.tif',
 'data/1214499_mosaic.tif']

## Classify Scenes

In this section, we use the ipynb module to import classification functionality from the [drc_roads_classification](drc_roads_classification.ipynb) notebook.

In [35]:
X_training, y_training = load_training_data()

In [ ]:
with Timer():
    classified_bands = [classify_forest(img, X_training, y_training)
                        for img in strip_mosaics]

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0 seconds
11 seconds


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0 seconds
13 seconds


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

0 seconds
14 seconds


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0 seconds
15 seconds


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0 seconds


In [ ]:
import numpy as np

In [ ]:
# We only have two values, 0 and 1. Convert to uint8 for memory. 
classified_bands = [band.astype(np.uint8) for band in classified_bands]

### Visualize Classified Scenes

In [ ]:
import visual

In [ ]:
# Decimate classified arrays for memory conservation
def decimate(arry, num=4):
    return arry[::num, ::num].copy()

for classified_band, strip_id in zip(classified_bands, strip_ids):
    visual.plot_image(classified_band_to_rgb(decimate(classified_band)),
                      title='Classified Image ({})'.format(strip_id),
                      figsize=(10, 10))

In looking at these classified images, the results of classification actually look really good. The issue with the ones that are classified as almost entirely non-forest is that they are primarily covered by clouds. Clouds are definately an issue here. It is tempting to dive into (1) using UDM to hopefully be able to filter out images that are primarily clouds and (2) increasing the date range to get more images after the new roads were built. But for now, let's forge on and see what temporal classification can get us.

## Temporal Classification

Lets look at the time series information for each pixel and see if we can identify true change

In [ ]:
def get_strip_date(strip_id):
    strip_scenes = overlapping_scenes[overlapping_scenes.strip_id == strip_id]
    strip_dates = set([a.date() for a in strip_scenes.acquired.tolist()])
    assert len(strip_dates) == 1
    return strip_dates.pop()

test_strip_id = '943459'
get_strip_date(test_strip_id)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# get time series for each unmasked pixel
pixel_timeseries = np.ma.compress_cols(np.ma.dstack(classified_bands).transpose(2, 0, 1).reshape(9, -1)).transpose(1,0)
pixel_timeseries.shape

In [ ]:
def make_sample(pixels, count):
    pixel_sample = pixels.copy()
    np.random.shuffle(pixel_sample) # only shuffles along the first axis
    pixel_sample = pixel_sample[:count, :]
    return pixel_sample

make_sample(pixel_timeseries, 6).shape

In [ ]:
def make_same_size_samples(list_of_pixel_sets):
    sample_len = min([p.shape[0] for p in list_of_pixel_sets])

    def sample_pixels(pixels):
        if pixels.shape[0] > sample_len:
            pixel_sample = make_sample(pixels, sample_len)
        else:
            pixel_sample = pixels
        return pixel_sample
    
    return [sample_pixels(p)
            for p in list_of_pixel_sets]

forest_pixels = pixel_timeseries[pixel_timeseries[:,0] == 0]
non_forest_pixels = pixel_timeseries[pixel_timeseries[:,0] == 1]
forest_pixels_sample, non_forest_pixels_sample = make_same_size_samples([forest_pixels, non_forest_pixels])
print(forest_pixels_sample.shape)
print(non_forest_pixels_sample.shape)

In [ ]:
forest_plot_sample = make_sample(forest_pixels_sample, 6)
non_forest_plot_sample = make_sample(non_forest_pixels_sample, 6)

In [ ]:
plt.figure()
for i, pixel in enumerate(forest_plot_sample):
    plt.plot(np.array(range(len(pixel))), pixel, 'o--', label=i)
# plt.legend(loc='upper left')
plt.show()

In [ ]:
plt.figure()
for i, pixel in enumerate(non_forest_plot_sample):
    plt.plot(np.array(range(len(pixel))), pixel, 'o--', label=i)
# plt.legend(loc='upper left')
plt.show()